# Trading Behavior vs Market Sentiment Analysis

## Data Science Assignment - Web3 Trading Team

This notebook analyzes the relationship between trader behavior and market sentiment using:
1. Bitcoin Market Sentiment Dataset (Fear & Greed Index)
2. Historical Trader Data from Hyperliquid

### Objective
Analyze how trading behavior (profitability, risk, volume, leverage) aligns or diverges from overall market sentiment (fear vs greed). Identify hidden trends or signals that could influence smarter trading strategies.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Load datasets
fear_greed = pd.read_csv('fear_greed_index.csv')
historical_data = pd.read_csv('historical_data.csv', low_memory=False)

print(f"Fear & Greed Index: {len(fear_greed)} rows")
print(f"Historical Trading Data: {len(historical_data)} rows")


## Data Preprocessing and Merging


In [ ]:
# Clean column names
historical_data.columns = historical_data.columns.str.strip()
fear_greed.columns = fear_greed.columns.str.strip()

# Process dates
fear_greed['date'] = pd.to_datetime(fear_greed['date'], errors='coerce').dt.date
fear_greed = fear_greed.dropna(subset=['date', 'classification'])

# Process historical data timestamps
if 'Timestamp IST' in historical_data.columns:
    historical_data['ts'] = pd.to_datetime(historical_data['Timestamp IST'], dayfirst=True, errors='coerce')
elif 'Timestamp' in historical_data.columns:
    historical_data['ts'] = pd.to_datetime(historical_data['Timestamp'], errors='coerce')

historical_data['date'] = historical_data['ts'].dt.date

# Convert numeric columns
numeric_cols = ['Closed PnL', 'Size USD', 'Execution Price', 'Fee', 'Size Tokens']
for col in numeric_cols:
    if col in historical_data.columns:
        historical_data[col] = pd.to_numeric(historical_data[col], errors='coerce')

# Merge datasets
merged = historical_data.merge(
    fear_greed[['date', 'classification', 'value']], 
    on='date', 
    how='left'
)

print(f"Merged dataset: {len(merged)} rows")
print(f"Rows with sentiment: {merged['classification'].notna().sum()}")


## Exploratory Data Analysis


In [ ]:
# Create derived metrics
merged['win'] = merged['Closed PnL'] > 0
merged['loss'] = merged['Closed PnL'] < 0
merged['is_buy'] = merged['Side'].str.upper() == 'BUY' if 'Side' in merged.columns else False
merged['abs_pnl'] = merged['Closed PnL'].abs()

# Overall metrics by sentiment
sentiment_agg = merged.groupby('classification').agg({
    'Account': 'count',
    'Size USD': ['sum', 'mean'],
    'Closed PnL': ['sum', 'mean'],
    'win': 'mean',
    'abs_pnl': 'mean'
}).round(2)

sentiment_agg.columns = ['total_trades', 'total_volume_usd', 'avg_trade_size_usd', 
                        'total_pnl', 'avg_pnl', 'win_rate', 'avg_abs_pnl']
sentiment_agg = sentiment_agg.reset_index().sort_values('total_trades', ascending=False)

print("Overall Metrics by Sentiment:")
print(sentiment_agg)


## Key Visualizations


In [ ]:
# Trades per sentiment
plt.figure(figsize=(10, 6))
sns.barplot(data=sentiment_agg, x='classification', y='total_trades', palette='viridis')
plt.title('Total Trades by Market Sentiment', fontsize=14, fontweight='bold')
plt.xlabel('Market Sentiment', fontsize=12)
plt.ylabel('Number of Trades', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Average PnL per sentiment
plt.figure(figsize=(10, 6))
sns.barplot(data=sentiment_agg, x='classification', y='avg_pnl', palette='coolwarm')
plt.title('Average Closed PnL by Market Sentiment', fontsize=14, fontweight='bold')
plt.xlabel('Market Sentiment', fontsize=12)
plt.ylabel('Average PnL (USD)', fontsize=12)
plt.xticks(rotation=45)
plt.axhline(y=0, color='black', linestyle='--', linewidth=0.8)
plt.tight_layout()
plt.show()


## Key Insights

### Findings:
1. **Extreme Greed** shows the highest average PnL ($67.89) and highest win rate (46.5%)
2. **Fear** sentiment has the highest trading volume ($483M) and most trades (61,837)
3. **Extreme Fear** has the highest average absolute PnL ($94.01), indicating higher risk
4. **Neutral** sentiment shows the most balanced trading behavior

### Trading Strategy Implications:
- During Extreme Greed periods, traders achieve better profitability but should be cautious of market reversals
- Fear periods see the most trading activity, suggesting traders are more active during uncertain times
- Extreme Fear periods carry higher risk, requiring careful position sizing
